In [106]:
import numpy as np
import pandas as pd
from datetime import date, datetime
import glob
import sys
from operator import itemgetter

## Wins & Losses of All Players in ATP Matches until today

In [54]:
def get_atpWinsLosses(_matches):
    """
    returns the wins and losses of all players 
    until today (April2022)
    and save them in a csv file
    """
    #matches = _matches [(_matches['tourney_level'] == 'A')]
    finals = _matches[(_matches['round'] == 'F')]
    semifinals = _matches[(_matches['round'] == 'SF')]
    
    #get players who won a title (final)
    titles_group = finals.groupby('winner_name').size()
    #get players who reach a final match - whether won or lost
    finals_group = semifinals.groupby('winner_name').size()
    
    #get all players who won a game or lost a game so as to count wins/losses
    w_group = _matches.groupby('winner_name').size()
    l_group = _matches.groupby('loser_name').size()
    
    scores = pd.DataFrame({'total_wins': w_group, 'total_losses': l_group}).fillna(0)
    scores[['total_wins', 'total_losses']] = scores[['total_wins', 'total_losses']].astype(int)
    
    scores = scores.reindex(['total_wins', 'total_losses'], axis=1)
    
    scores['total_matches'] = scores['total_wins'] + scores['total_losses']
    scores['perc_of_total_wins'] = np.round(scores['total_wins']*100/scores['total_matches'],2)
    
    scores.index.name = 'player_name'
    
    scores = scores.join(pd.DataFrame(finals_group, columns = ['finals'],)).fillna(0)
    scores = scores.join(pd.DataFrame(titles_group, columns = ['titles'],)).fillna(0)
    
    scores['titles'] = scores['titles'].astype('int')
    scores['finals'] = scores['finals'].astype('int')
    
    scores = scores.sort_values(['titles', 'total_wins'], ascending=False)
    scores.to_csv('atp/atp_allwinslosses.csv') 
    #print(scores.shape[0])
    #print(scores.to_csv(sys.stdout,index=True))
    return scores 

In [55]:
scores = get_atpWinsLosses(atpmatches)
scores.head(10)

,total_wins,total_losses,total_matches,perc_of_total_wins,finals,titles
player_name,,,,,,
Roger Federer,1170,250,1420,82.39,151,102
Rafael Nadal,975,198,1173,83.12,121,87
Novak Djokovic,927,179,1106,83.82,117,86
Andy Murray,643,209,852,75.47,64,45
Andy Roddick,568,188,756,75.13,50,31
Lleyton Hewitt,506,215,721,70.18,39,28
David Ferrer,691,364,1055,65.50,52,27
Juan Martin del Potro,415,155,570,72.81,33,22
Nikolay Davydenko,463,298,761,60.84,27,21


In [56]:
#playerscopy = players.copy()

scores = scores.sort_values(['player_name'])
# uncomment the above line only for the first time 
# do not re-run it !! 
#scores = scores.reset_index(level=0)
players_atpComp = scores['player_name'].tolist()

In [57]:
scores.head()

,player_name,total_wins,total_losses,total_matches,perc_of_total_wins,finals,titles
0,Abdulla Hajji,0,3,3,0.00,0,0
1,Adam Chadaj,1,1,2,50.00,0,0
2,Adam Kennedy,0,3,3,0.00,0,0
3,Adam Pavlasek,5,9,14,35.71,0,0
4,Adrian Andreev,2,6,8,25.00,0,0


In [58]:
players.head()

,player_id,name,name_first,name_last,hand,dob,ioc,height
50201,205167,Rafa Angel 1,Rafa Angel,1,U,19800723,NGR,NaN
17180,117196,NaN,NaN,A Cantacuzene,U,NaN,ROU,NaN
17091,117107,NaN,NaN,A Riches,U,NaN,EGY,NaN
31495,131511,Bengt Aaberg,Bengt,Aaberg,U,19451216,SWE,NaN
51815,206781,Ludvig Aaes,Ludvig,Aaes,R,19980219,DEN,NaN


An inner join combines two DataFrames based on a join key and returns a new DataFrame that contains only those rows that have matching values in both of the original DataFrames.

In [60]:
merged_ = pd.merge(left=players, right=scores, left_on='name', right_on='player_name')

players_atp = merged_[['player_id', 'player_name','name_first','name_last','hand',
                      'dob','ioc','height','total_wins','total_losses','total_matches',
                      'perc_of_total_wins','finals','titles']]
players_atp = players_atp.sort_values(['total_wins'],ascending=False)
players_atp.head(10)

,player_id,player_name,name_first,name_last,hand,dob,ioc,height,total_wins,total_losses,total_matches,perc_of_total_wins,finals,titles
380,103819,Roger Federer,Roger,Federer,R,19810808,SUI,185.0,1170,250,1420,82.39,151,102
903,104745,Rafael Nadal,Rafael,Nadal,L,19860603,ESP,185.0,975,198,1173,83.12,121,87
309,104925,Novak Djokovic,Novak,Djokovic,R,19870522,SRB,188.0,927,179,1106,83.82,117,86
385,103970,David Ferrer,David,Ferrer,R,19820402,ESP,175.0,691,364,1055,65.50,52,27
898,104918,Andy Murray,Andy,Murray,R,19870515,GBR,190.0,643,209,852,75.47,64,45
107,104607,Tomas Berdych,Tomas,Berdych,R,19850917,CZE,196.0,599,319,918,65.25,31,14
1083,104053,Andy Roddick,Andy,Roddick,R,19820830,USA,188.0,568,188,756,75.13,50,31
433,104755,Richard Gasquet,Richard,Gasquet,R,19860618,FRA,185.0,534,318,852,62.68,30,14
1352,104269,Fernando Verdasco,Fernando,Verdasco,L,19831115,ESP,188.0,524,420,944,55.51,22,7
1078,103990,Tommy Robredo,Tommy,Robredo,R,19820501,ESP,180.0,511,339,850,60.12,21,12


In [61]:
players_atp.to_csv('atp/atp_players_update.csv')

## Favorable Surfaces

In [62]:
def get_atpWinsHardSurface(_matches):
    
    hard = _matches[(_matches['surface'] == 'Hard')]
    
    w_group = hard.groupby('winner_name').size()
    l_group = hard.groupby('loser_name').size()
    
    surface = pd.DataFrame({'wins_hard': w_group, 'losses_hard': l_group}).fillna(0)
    surface[['wins_hard', 'losses_hard']] = surface[['wins_hard', 'losses_hard']].astype(int)
    
    surface = surface.reindex(['wins_hard', 'losses_hard'], axis=1)
    
    surface['matches_hard'] = surface['wins_hard'] + surface['losses_hard']
    surface['percentage_hard'] = np.round(surface['wins_hard']*100/surface['matches_hard'],2)
    
    surface.index.name = 'player_name'
    
    surface.to_csv('atp/atp_WinsHardSurface.csv')
    return surface

In [63]:
hard = get_atpWinsHardSurface(atpmatches)
hard = hard.sort_values(['wins_hard', 'percentage_hard'], ascending=False)
hard.head()

,wins_hard,losses_hard,matches_hard,percentage_hard
player_name,,,,
Roger Federer,736,147,883,83.35
Novak Djokovic,594,109,703,84.50
Rafael Nadal,466,131,597,78.06
Andy Murray,438,140,578,75.78
Andy Roddick,401,127,528,75.95


In [64]:
def get_atpWinsClaySurface(_matches):
    
    clay = _matches[(_matches['surface'] == 'Clay')]
    
    w_group = clay.groupby('winner_name').size()
    l_group = clay.groupby('loser_name').size()
    
    surface = pd.DataFrame({'wins_clay': w_group, 'losses_clay': l_group}).fillna(0)
    surface[['wins_clay', 'losses_clay']] = surface[['wins_clay', 'losses_clay']].astype(int)
    
    surface = surface.reindex(['wins_clay', 'losses_clay'], axis=1)
    
    surface['matches_clay'] = surface['wins_clay'] + surface['losses_clay']
    surface['percentage_clay'] = np.round(surface['wins_clay']*100/surface['matches_clay'],2)
    
    surface.index.name = 'player_name'
    
    surface.to_csv('atp/atp_WinsClaySurface.csv')
    return surface

In [65]:
clay = get_atpWinsClaySurface(atpmatches)
clay = clay.sort_values(['wins_clay', 'percentage_clay'], ascending=False)
clay.head()

,wins_clay,losses_clay,matches_clay,percentage_clay
player_name,,,,
Rafael Nadal,438,43,481,91.06
David Ferrer,310,137,447,69.35
Nicolas Almagro,263,136,399,65.91
Tommy Robredo,247,123,370,66.76
Novak Djokovic,227,50,277,81.95


In [66]:
def get_atpWinsGrassSurface(_matches):
    
    grass = _matches[(_matches['surface'] == 'Grass')]
    
    w_group = grass.groupby('winner_name').size()
    l_group = grass.groupby('loser_name').size()
    
    surface = pd.DataFrame({'wins_grass': w_group, 'losses_grass': l_group}).fillna(0)
    surface[['wins_grass', 'losses_grass']] = surface[['wins_grass', 'losses_grass']].astype(int)
    
    surface = surface.reindex(['wins_grass', 'losses_grass'], axis=1)
    
    surface['matches_grass'] = surface['wins_grass'] + surface['losses_grass']
    surface['percentage_grass'] = np.round(surface['wins_grass']*100/surface['matches_grass'],2)
    
    surface.index.name = 'player_name'
    
    surface.to_csv('atp/atp_WinsGrassSurface.csv')
    return surface

In [67]:
grass = get_atpWinsGrassSurface(atpmatches)
grass = grass.sort_values(['wins_grass', 'percentage_grass'], ascending=False)
grass.head()

,wins_grass,losses_grass,matches_grass,percentage_grass
player_name,,,,
Roger Federer,187,27,214,87.38
Lleyton Hewitt,103,30,133,77.44
Andy Murray,102,23,125,81.60
Novak Djokovic,101,16,117,86.32
Feliciano Lopez,83,45,128,64.84


In [68]:
def get_atpWinsCarpetSurface(_matches):
    
    carpet = _matches[(_matches['surface'] == 'Carpet')]
    
    w_group = carpet.groupby('winner_name').size()
    l_group = carpet.groupby('loser_name').size()
    
    surface = pd.DataFrame({'wins_carpet': w_group, 'losses_carpet': l_group}).fillna(0)
    surface[['wins_carpet', 'losses_carpet']] = surface[['wins_carpet', 'losses_carpet']].astype(int)
    
    surface = surface.reindex(['wins_carpet', 'losses_carpet'], axis=1)
    
    surface['matches_carpet'] = surface['wins_carpet'] + surface['losses_carpet']
    surface['percentage_carpet'] = np.round(surface['wins_carpet']*100/surface['matches_carpet'],2)
    
    surface.index.name = 'player_name'
    
    surface.to_csv('atp/atp_WinsCarpetSurface.csv')
    return surface

In [69]:
carpet = get_atpWinsCarpetSurface(atpmatches)
carpet = carpet.sort_values(['wins_carpet', 'percentage_carpet'], ascending=False)
carpet.head()

,wins_carpet,losses_carpet,matches_carpet,percentage_carpet
player_name,,,,
Ivan Ljubicic,40,22,62,64.52
Marat Safin,38,14,52,73.08
Roger Federer,37,13,50,74.00
Mikhail Youzhny,31,17,48,64.58
Nikolay Davydenko,29,14,43,67.44


### Concatenate all types of surfaces in one dataframe

I use concat so as to keep all players in the new dataframe. 

The merge() function performs an inner join by default, so only the indexes that appear in both DataFrames are kept.

The join() function performs a left join by default, so each of the indexes in the first DataFrame are kept.

The concat() function performs an outer join by default, so each index value from each DataFrame is kept.

In [70]:
hard_carpet = pd.concat([hard, carpet],axis=1)
hard_carpet['matches1'] = hard_carpet['matches_hard'] + hard_carpet['matches_carpet']

grass_clay = pd.concat([grass, clay], axis=1)
grass_clay['matches2'] = grass_clay['matches_grass'] + grass_clay['matches_clay']

In [71]:
allsurfaces = pd.concat([hard_carpet, grass_clay], axis=1)
allsurfaces['matches'] = allsurfaces['matches1'] + allsurfaces['matches2']

In [72]:
allsurfaces = allsurfaces[['matches', 'percentage_hard', 'percentage_carpet', 'percentage_grass', 'percentage_clay']]
allsurfaces.fillna(0, inplace=True)
allsurfaces.to_csv('atp/atp_allsurfaces_PERCENTAGE_.csv')

In [73]:
allsurfaces = allsurfaces.sort_values(['matches'], ascending=False)
allsurfaces.head()

,matches,percentage_hard,percentage_carpet,percentage_grass,percentage_clay
Roger Federer,1420.0,83.35,74.00,87.38,76.92
Rafael Nadal,1173.0,78.06,16.67,78.65,91.06
Novak Djokovic,1106.0,84.50,55.56,86.32,81.95
David Ferrer,1055.0,63.36,43.75,61.76,69.35
Fernando Verdasco,944.0,52.78,57.14,53.26,59.46


In [74]:
allsurfaces = allsurfaces.reset_index(level=0)

In [75]:
allsurfaces.rename(columns = {'index': 'player_name'}, inplace=True)
allsurfaces.head()

,player_name,matches,percentage_hard,percentage_carpet,percentage_grass,percentage_clay
0,Roger Federer,1420.0,83.35,74.00,87.38,76.92
1,Rafael Nadal,1173.0,78.06,16.67,78.65,91.06
2,Novak Djokovic,1106.0,84.50,55.56,86.32,81.95
3,David Ferrer,1055.0,63.36,43.75,61.76,69.35
4,Fernando Verdasco,944.0,52.78,57.14,53.26,59.46


In [76]:
allsurfaces_merged = pd.merge(left=players_atp, right=allsurfaces, left_on='player_name', right_on='player_name')

In [77]:
allsurfaces_merged.head()

,player_id,player_name,name_first,name_last,hand,dob,ioc,height,total_wins,total_losses,total_matches,perc_of_total_wins,finals,titles,matches,percentage_hard,percentage_carpet,percentage_grass,percentage_clay
0,103819,Roger Federer,Roger,Federer,R,19810808,SUI,185.0,1170,250,1420,82.39,151,102,1420.0,83.35,74.00,87.38,76.92
1,104745,Rafael Nadal,Rafael,Nadal,L,19860603,ESP,185.0,975,198,1173,83.12,121,87,1173.0,78.06,16.67,78.65,91.06
2,104925,Novak Djokovic,Novak,Djokovic,R,19870522,SRB,188.0,927,179,1106,83.82,117,86,1106.0,84.50,55.56,86.32,81.95
3,103970,David Ferrer,David,Ferrer,R,19820402,ESP,175.0,691,364,1055,65.50,52,27,1055.0,63.36,43.75,61.76,69.35
4,104918,Andy Murray,Andy,Murray,R,19870515,GBR,190.0,643,209,852,75.47,64,45,852.0,75.78,72.73,81.60,68.84


In [78]:
allsurfaces_merged.to_csv('atp/atp_players_update.csv')

## Age & Difference in Age per Match

In [81]:
agedifferences = atpmatches.copy()
agedifferences['age_difference'] = round(abs(agedifferences['winner_age'] - agedifferences['loser_age']))
agedifferences.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,age_difference
591,2000-339,Adelaide,Hard,32,A,20000103,1,102358,1.0,NaN,...,25.0,13.0,9.0,2.0,4.0,4.0,2606.0,56.0,805.0,4.0
592,2000-339,Adelaide,Hard,32,A,20000103,2,103819,NaN,NaN,...,13.0,12.0,8.0,0.0,3.0,64.0,749.0,91.0,525.0,6.0
593,2000-339,Adelaide,Hard,32,A,20000103,3,102998,NaN,NaN,...,49.0,22.0,16.0,4.0,5.0,58.0,803.0,105.0,449.0,6.0
594,2000-339,Adelaide,Hard,32,A,20000103,4,103206,7.0,NaN,...,12.0,8.0,8.0,1.0,6.0,27.0,1298.0,54.0,845.0,2.0
595,2000-339,Adelaide,Hard,32,A,20000103,5,102796,3.0,NaN,...,25.0,16.0,10.0,7.0,10.0,15.0,1748.0,154.0,297.0,2.0


In [82]:
agedifferences.to_csv('atp/atp_matches_update.csv')

## Best-of-X

There are some matches, that although matches are best-of-3, only the Final is best-of-5. 

In [34]:
tournaments = atpmatches[['tourney_id', 'tourney_name','best_of']]
tour = tournaments.groupby(['tourney_id', 'tourney_name']).mean()
tour.to_csv('atp/atp_bestOf.csv')
tour.head(10)

,,best_of
tourney_id,tourney_name,
2000-301,Auckland,3.000000
2000-306,St. Poelten,3.000000
2000-308,Munich,3.000000
2000-311,Queen's Club,3.000000
2000-314,Gstaad,3.000000
2000-315,Newport,3.000000
2000-316,Bastad,3.000000
2000-317,Amsterdam,3.066667
2000-319,Kitzbuhel,3.000000


## Home Advantage

In [97]:
atpmatches.head()

home = atpmatches.copy()

In [98]:
conditions = [
    (home['tourney_name'] == 'Adelaide'), (home['tourney_name'] == 'Doha'),
    (home['tourney_name'] == 'Chennai'), (home['tourney_name'] == 'Auckland'),
    (home['tourney_name'] == 'Sydney'), (home['tourney_name'] == 'Australian Open'),
    (home['tourney_name'] == 'San Jose'), (home['tourney_name'] == 'Dubai'),
    (home['tourney_name'] == 'Marseille'), (home['tourney_name'] == 'Memphis'),
    (home['tourney_name'] == 'Rotterdam'), (home['tourney_name'] == 'London'),
    (home['tourney_name'] == 'Mexico City'), (home['tourney_name'] == 'Copenhagen'),
    (home['tourney_name'] == 'Delray Beach'), (home['tourney_name'] == 'Santiago'),
    (home['tourney_name'] == 'Bogota'), (home['tourney_name'] == 'Scottsdale'),
    (home['tourney_name'] == 'Indian Wells Masters'), 
    (home['tourney_name'] == 'Miami Masters'), (home['tourney_name'] == 'Casablanca'),
    (home['tourney_name'] == 'Atlanta'), (home['tourney_name'] == 'Estoril'),
    (home['tourney_name'] == 'Monte Carlo Masters'), (home['tourney_name'] == 'Barcelona'),
    (home['tourney_name'] == 'Munich'), (home['tourney_name'] == 'Mallorca'),
    (home['tourney_name'] == 'Orlando'), (home['tourney_name'] == 'Rome Masters'),
    (home['tourney_name'] == 'Hamburg Masters'), (home['tourney_name'] == 'Dusseldorf'),
    (home['tourney_name'] == 'St. Poelten'), (home['tourney_name'] == 'Roland Garros'),
    (home['tourney_name'] == "Queen's Club"), (home['tourney_name'] == 'Halle'),
    (home['tourney_name'] == 's Hertogenbosch'), (home['tourney_name'] == 'Nottingham'),
    (home['tourney_name'] == 'Wimbledon'), (home['tourney_name'] == 'Gstaad'),
    (home['tourney_name'] == 'Newport'), (home['tourney_name'] == 'Bastad'),
    (home['tourney_name'] == 'Amsterdam'), (home['tourney_name'] == 'Stuttgart Outdoor'),
    (home['tourney_name'] == 'Umag'), (home['tourney_name'] == 'Kitzbuhel'),  
    (home['tourney_name'] == 'Los Angeles'), (home['tourney_name'] == 'San Marino'),
    (home['tourney_name'] == 'Canada Masters'), (home['tourney_name'] == 'Cincinnati Masters'),
    (home['tourney_name'] == 'Washington'), (home['tourney_name'] == 'Indianapolis'),
    (home['tourney_name'] == 'Long Island'), (home['tourney_name'] == 'US Open'),
    (home['tourney_name'] == 'Tashkent'), (home['tourney_name'] == 'Bucharest'), 
    (home['tourney_name'] == 'Sydney Olympics'), (home['tourney_name'] == 'Palermo'), 
    (home['tourney_name'] == 'Hong Kong'), (home['tourney_name'] == 'Tokyo'), 
    (home['tourney_name'] == 'Vienna'), (home['tourney_name'] == 'Toulouse'), 
    (home['tourney_name'] == 'Shanghai'), (home['tourney_name'] == 'Basel'), 
    (home['tourney_name'] == 'Moscow'), (home['tourney_name'] == 'Stuttgart Masters'),     
    (home['tourney_name'] == 'Lyon'), (home['tourney_name'] == 'St. Petersburg'), 
    (home['tourney_name'] == 'Paris Masters'), (home['tourney_name'] == 'Stockholm'),
    (home['tourney_name'] == 'Brighton'), (home['tourney_name'] == 'Masters Cup'),
    (home['tourney_name'] == 'Milan'), (home['tourney_name'] == 'Vina del Mar'),
    (home['tourney_name'] == 'Buenos Aires'), (home['tourney_name'] == 'Acapulco'),
    (home['tourney_name'] == 'Houston'), (home['tourney_name'] == 'Stuttgart'),
    (home['tourney_name'] == 'Sopot'), (home['tourney_name'] == 'Costa Do Sauipe'),
    (home['tourney_name'] == 'Amersfoort'), (home['tourney_name'] == 'Madrid Masters'),
    (home['tourney_name'] == 'Valencia'), (home['tourney_name'] == 'Bangkok'),
    (home['tourney_name'] == 'Metz'), (home['tourney_name'] == 'Athens Olympics'),
    (home['tourney_name'] == 'Beijing'), (home['tourney_name'] == 'New Haven'),
    (home['tourney_name'] == 'Ho Chi Minh City'), (home['tourney_name'] == 'Zagreb'),
    (home['tourney_name'] == 'Las Vegas'), (home['tourney_name'] == 'Poertschach'),
    (home['tourney_name'] == 'Mumbai'), (home['tourney_name'] == 'Warsaw'), 
    (home['tourney_name'] == 'Beijing Olympics'), (home['tourney_name'] == 'Brisbane'),
    (home['tourney_name'] == 'Johannesburg'), (home['tourney_name'] == 'Belgrade'),
    (home['tourney_name'] == 'Eastbourne'), (home['tourney_name'] == 'Hamburg'),
    (home['tourney_name'] == 'Kuala Lumpur'), (home['tourney_name'] == 'Shanghai Masters'),
    (home['tourney_name'] == 'Nice'), (home['tourney_name'] == 'Montpellier'),
    (home['tourney_name'] == 'Winston-Salem'), (home['tourney_name'] == 'Sao Paulo'), 
    (home['tourney_name'] == 'London Olympics'), (home['tourney_name'] == 'Rio de Janeiro'),
    (home['tourney_name'] == 'Shenzhen'), (home['tourney_name'] == 'Quito'),
    (home['tourney_name'] == 'Istanbul'), (home['tourney_name'] == 'Geneva'),
    (home['tourney_name'] == 'Sofia'), (home['tourney_name'] == 'Marrakech'),
    (home['tourney_name'] == 'Los Cabos'), (home['tourney_name'] == 'Rio Olympics'),
    (home['tourney_name'] == 'Chengdu'), (home['tourney_name'] == 'Antwerp'),
    (home['tourney_name'] == 'Budapest'), (home['tourney_name'] == 'Antalya'),
    (home['tourney_name'] == 'NextGen Finals'), (home['tourney_name'] == 'Pune'),
    (home['tourney_name'] == 'New York'), (home['tourney_name'] == 'Cordoba'),
    (home['tourney_name'] == 'Zhuhai'), (home['tourney_name'] == 'Atp Cup'), 
    (home['tourney_name'] == 'ATP Rio de Janeiro'), (home['tourney_name'] == 'Us Open'),
    (home['tourney_name'] == 'St Petersburg'), (home['tourney_name'] == 'Cologne 1'),
    (home['tourney_name'] == 'Sardinia'), (home['tourney_name'] == 'Cologne 2'),
    (home['tourney_name'] == 'Nur-Sultan'), (home['tourney_name'] == 'San Diego'),
    (home['tourney_name'] == 'Great Ocean Road Open'), (home['tourney_name'] == 'Murray River Open'),
    (home['tourney_name'] == 'Singapore'), (home['tourney_name'] == 'Marbella'), 
    (home['tourney_name'] == 'Cagliari'), (home['tourney_name'] == 'Parma'), 
    (home['tourney_name'] == 'Belgrade 2'), (home['tourney_name'] == 'Tokyo Olympics')    
]

values = [ 
    'AUS', 'QAT', 'IND', 'NZL', 'AUS', 'AUS', 'CRI', 'UAE', 'FRA', 'USA', 
    'NDL', 'GBR', 'MEX', 'DNK', 'USA', 'CHI', 'COL', 'USA', 'USA', 'USA', 
    'MAR', 'USA', 'PRT', 'MON', 'ESP', 'GER', 'ESP', 'USA', 'ITA', 'GER', 
    'GER', 'AUT', 'FRA', 'GBR', 'GER', 'NLD', 'GBR', 'GBR', 'SUI', 'GBR', 
    'SWE', 'NLD', 'GER', 'CRO', 'AUT', 'USA', 'SMR', 'CAN', 'USA', 'USA', 
    'USA', 'USA', 'USA', 'UZB', 'ROU', 'AUS', 'ITA', 'CHN', 'JPN', 'AUT', 
    'FRA', 'CHN', 'SUI', 'RUS', 'GER', 'FRA', 'RUS', 'FRA', 'SWE', 'GBR', 
    'USA', 'ITA', 'CHI', 'ARG', 'MEX', 'USA', 'GER', 'POL', 'BRA', 'NDL', 
    'ESP', 'ESP', 'THA', 'FRA', 'GRE', 'CHN', 'USA', 'VNM', 'CRO', 'USA',
    'AUT', 'IND', 'POL', 'CHN', 'AUS', 'RSA', 'SRB', 'GBR', 'GER', 'MAS', 
    'CHN', 'FRA', 'FRA', 'USA', 'BRA', 'GBR', 'BRA', 'CHN', 'ECU', 'TUR', 
    'SUI', 'BGR', 'MAR', 'MEX', 'BRA', 'CHN', 'BEL', 'HUN', 'TUR', 'ITA', 
    'IND', 'USA', 'ESP', 'CHN', 'AUS', 'BRA', 'USA', 'RUS', 'GER', 'ITA', 
    'GER', 'KAZ', 'AUS', 'AUS', 'SGP', 'ESP', 'ITA', 'ITA', 'SRB', 'JPN', 
    'USA'
]

In [99]:
home['location_country'] = np.select(conditions, values)
home.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,location_country
591,2000-339,Adelaide,Hard,32,A,20000103,1,102358,1.0,NaN,...,25.0,13.0,9.0,2.0,4.0,4.0,2606.0,56.0,805.0,AUS
592,2000-339,Adelaide,Hard,32,A,20000103,2,103819,NaN,NaN,...,13.0,12.0,8.0,0.0,3.0,64.0,749.0,91.0,525.0,AUS
593,2000-339,Adelaide,Hard,32,A,20000103,3,102998,NaN,NaN,...,49.0,22.0,16.0,4.0,5.0,58.0,803.0,105.0,449.0,AUS
594,2000-339,Adelaide,Hard,32,A,20000103,4,103206,7.0,NaN,...,12.0,8.0,8.0,1.0,6.0,27.0,1298.0,54.0,845.0,AUS
595,2000-339,Adelaide,Hard,32,A,20000103,5,102796,3.0,NaN,...,25.0,16.0,10.0,7.0,10.0,15.0,1748.0,154.0,297.0,AUS


In [101]:
home = home[['tourney_id', 'tourney_name', 'location_country', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'
       ]]
home.head()

,tourney_id,tourney_name,location_country,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
591,2000-339,Adelaide,AUS,Hard,32,A,20000103,1,102358,1.0,...,37.0,25.0,13.0,9.0,2.0,4.0,4.0,2606.0,56.0,805.0
592,2000-339,Adelaide,AUS,Hard,32,A,20000103,2,103819,NaN,...,15.0,13.0,12.0,8.0,0.0,3.0,64.0,749.0,91.0,525.0
593,2000-339,Adelaide,AUS,Hard,32,A,20000103,3,102998,NaN,...,59.0,49.0,22.0,16.0,4.0,5.0,58.0,803.0,105.0,449.0
594,2000-339,Adelaide,AUS,Hard,32,A,20000103,4,103206,7.0,...,22.0,12.0,8.0,8.0,1.0,6.0,27.0,1298.0,54.0,845.0
595,2000-339,Adelaide,AUS,Hard,32,A,20000103,5,102796,3.0,...,40.0,25.0,16.0,10.0,7.0,10.0,15.0,1748.0,154.0,297.0


In [102]:
# 1 is for the win, 2 is for lose for games in homecountry
home['home_advantage'] = np.select([(home['winner_ioc'] == home['location_country'])], [1])
home['home_advantage'] = np.select([(home['loser_ioc'] == home['location_country'])], [2])

In [104]:
home.to_csv('atp/atp_matches_update.csv')
home.head()

,tourney_id,tourney_name,location_country,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,home_advantage
591,2000-339,Adelaide,AUS,Hard,32,A,20000103,1,102358,1.0,...,25.0,13.0,9.0,2.0,4.0,4.0,2606.0,56.0,805.0,0
592,2000-339,Adelaide,AUS,Hard,32,A,20000103,2,103819,NaN,...,13.0,12.0,8.0,0.0,3.0,64.0,749.0,91.0,525.0,0
593,2000-339,Adelaide,AUS,Hard,32,A,20000103,3,102998,NaN,...,49.0,22.0,16.0,4.0,5.0,58.0,803.0,105.0,449.0,2
594,2000-339,Adelaide,AUS,Hard,32,A,20000103,4,103206,7.0,...,12.0,8.0,8.0,1.0,6.0,27.0,1298.0,54.0,845.0,2
595,2000-339,Adelaide,AUS,Hard,32,A,20000103,5,102796,3.0,...,25.0,16.0,10.0,7.0,10.0,15.0,1748.0,154.0,297.0,2


## Wins on specific Tournament

In [36]:
def wins_on_tour_per_player(winsOnTour, tourney):
    
    tourney_ = str(tourney)
    matches = winsOnTour[(winsOnTour['tourney_name'] == tourney_)]
    finals = matches[(matches['round'] == 'F')]
    semifinals = matches[(matches['round'] == 'SF')]
    
    #get players who won a title (final)
    titles_group = finals.groupby('winner_name').size()
    #get players who reach a final match - whether won or lost
    finals_group = semifinals.groupby('winner_name').size()
    
    #get all players who won a game or lost a game so as to count wins/losses
    w_group = matches.groupby('winner_name').size()
    l_group = matches.groupby('loser_name').size()
    
    scores = pd.DataFrame({'wins': w_group, 'losses': l_group}).fillna(0)
    scores[['wins', 'losses']] = scores[['wins', 'losses']].astype(int)
    
    scores = scores.reindex(['wins', 'losses'], axis=1)
    
    scores['matches'] = scores['wins'] + scores['losses']
    scores['percentage'] = np.round(scores['wins']*100/scores['matches'],2)
    
    scores.index.name = 'player_name'
    
    scores = scores.join(pd.DataFrame(finals_group, columns = ['finals'],)).fillna(0)
    scores = scores.join(pd.DataFrame(titles_group, columns = ['titles'],)).fillna(0)
    
    scores['titles'] = scores['titles'].astype('int')
    scores['finals'] = scores['finals'].astype('int')
    
    scores = scores.sort_values(['titles', 'wins'], ascending=False)
    return scores

In [37]:
tourlist = ['Adelaide', 'Doha', 'Chennai', 'Auckland', 'Sydney', 'Australian Open',
            'San Jose', 'Dubai', 'San Jose', 'Dubai', 'Marseille', 'Memphis',
            'Rotterdam', 'London', 'Mexico City', 'Copenhagen', 'Delray Beach',
            'Santiago', 'Bogota', 'Scottsdale', 'Indian Wells Masters', 'Miami Masters',
            'Casablanca', 'Atlanta', 'Estoril', 'Monte Carlo Masters', 'Barcelona', 
            'Munich', 'Mallorca', 'Orlando', 'Rome Masters', 'Hamburg Masters', 
            'Dusseldorf', 'St. Poelten', 'Roland Garros', "Queen's Club", 'Halle',
            's Hertogenbosch', 'Nottingham', 'Wimbledon', 'Gstaad', 'Newport', 'Bastad',
            'Amsterdam', 'Stuttgart Outdoor', 'Umag', 'Kitzbuhel', 'Los Angeles', 
            'San Marino', 'Canada Masters', 'Cincinnati Masters', 'Washington', 
            'Indianapolis', 'Long Island', 'US Open', 'Tashkent', 'Bucharest', 
            'Sydney Olympics', 'Palermo', 'Hong Kong', 'Tokyo', 'Vienna', 'Toulouse',
            'Shanghai', 'Basel', 'Moscow', 'Stuttgart Masters', 'Lyon', 'St. Petersburg',
            'Paris Masters', 'Stockholm', 'Brighton', 'Masters Cup', 'Milan', 
            'Vina del Mar', 'Buenos Aires', 'Acapulco', 'Houston', 'Stuttgart', 
            'Sopot', 'Costa Do Sauipe', 'Amersfoort', 'Madrid Masters', 'Valencia',
            'Bangkok', 'Metz', 'Athens Olympics', 'Ho Chi Minh City', 'Zagreb',
            'Las Vegas', 'Poertschach', 'Mumbai', 'Warsaw', 'Beijing Olympics', 
            'Brisbane', 'Johannesburg', 'Belgrade', 'Eastbourne', 'Hamburg', 
            'Kuala Lumpur', 'Shanghai Masters', 'Nice', 'Montpellier', 'Winston-Salem',
            'Sao Paulo', 'London Olympics', 'Rio de Janeiro', 'Shenzhen', 'Quito',
            'Istanbul', 'Geneva', 'Sofia', 'Marrakech', 'Los Cabos', 'Rio Olympics',
            'Chengdu', 'Antwerp', 'Budapest', 'Antalya', 'NextGen Finals', 'Pune', 
            'New York', 'Cordoba', 'Zhuhai', 'Atp Cup', 'ATP Rio de Janeiro', 'Us Open',
            'St Petersburg', 'Cologne 1', 'Sardinia', 'Cologne 2', 'Nur-Sultan', 
            'San Diego', 'Great Ocean Road Open', 'Murray River Open', 'Singapore', 
            'Marbella', 'Cagliari', 'Parma', 'Belgrade 2', 'Tokyo Olympics']

In [39]:
tour = atpmatches.copy()

for t in tourlist:
    #print(t)
    sc = wins_on_tour_per_player(tour, t)
    #print(sc)
    sc.to_csv('atp/atp_winslossesOn_' + t + '.csv')

## Head-to-Head

In [105]:
def geth2hforplayer(matches,name):
    """
    get all head-to-heads of the player
    returns list of all head-to-heads 
    like:
    if name = 'Roger Federer' then 
    the result ['Sergi Bruguera', 0, 1]
    means that Roger Federer had 0 wins 
    and Sergi Bruguera had 1 win.
    """
    
    matches = matches[(matches['winner_name'] == name) | (matches['loser_name'] == name)]
    h2hs = {}
    for index, match in matches.iterrows():
        if (match['winner_name'] == name):
            if (match['loser_name'] not in h2hs):
                h2hs[match['loser_name']] = {}
                h2hs[match['loser_name']]['l'] = 0
                h2hs[match['loser_name']]['w'] = 1
            else:
                h2hs[match['loser_name']]['w'] = h2hs[match['loser_name']]['w']+1
        elif (match['loser_name'] == name):
            if (match['winner_name'] not in h2hs):
                h2hs[match['winner_name']] = {}
                h2hs[match['winner_name']]['w'] = 0
                h2hs[match['winner_name']]['l'] = 1
            else:
                h2hs[match['winner_name']]['l'] = h2hs[match['winner_name']]['l']+1

    #create list
    h2hlist = []
    for k, v in h2hs.items():
        h2hlist.append([k, v['w'],v['l']])

    if (len(h2hlist) == 0):
        return ''
    else:
        return sorted(h2hlist, key=itemgetter(1,2))
        #for h2h in h2hlist:
        #    print(name+';'+h2h[0]+';'+str(h2h[1])+';'+str(h2h[2]))

In [107]:
# change the name based on the player you want
head2headresults = geth2hforplayer(atpmatches, 'Roger Federer')
head2headresults

[['Sergi Bruguera', 0, 1],
 ['Andrei Medvedev', 0, 1],
 ['Markus Hantschk', 0, 1],
 ['Richard Fromberg', 0, 1],
 ['Francisco Clavet', 0, 1],
 ['James Sekulov', 0, 1],
 ['Andrea Gaudenzi', 0, 1],
 ['Felix Mantilla', 0, 1],
 ['Evgeny Donskoy', 0, 1],
 ['Thanasi Kokkinakis', 0, 1],
 ['Andrey Rublev', 0, 1],
 ['Pablo Andujar', 0, 1],
 ['Felix Auger Aliassime', 0, 1],
 ['Patrick Rafter', 0, 2],
 ['Franco Squillari', 0, 2],
 ['Jens Knippschild', 1, 0],
 ['Jan Kroslak', 1, 0],
 ['Filip Dewulf', 1, 0],
 ['Fredrik Jonsson', 1, 0],
 ['Gianluca Pozzi', 1, 0],
 ['Justin Gimelstob', 1, 0],
 ['Daniel Nestor', 1, 0],
 ['Mikael Tillstrom', 1, 0],
 ['Karim Alami', 1, 0],
 ['Magnus Norman', 1, 0],
 ['Bob Bryan', 1, 0],
 ['Andrew Ilie', 1, 0],
 ['Stefano Galvani', 1, 0],
 ['David Sanchez', 1, 0],
 ['Pete Sampras', 1, 0],
 ['Attila Savolt', 1, 0],
 ['Hendrik Dreekmann', 1, 0],
 ['Jiri Vanek', 1, 0],
 ['Denis Golovanov', 1, 0],
 ['Alexander Waske', 1, 0],
 ['Andrei Stoliarov', 1, 0],
 ['Flavio Saretta', 1,

## Average number of aces per game (mean value) per player

In [108]:
aces = atpmatches.copy()

w_total = aces.groupby('winner_name')['w_ace'].mean()
w_ace = pd.DataFrame({'w_aces': w_total})

l_total = aces.groupby('loser_name')['l_ace'].mean()
l_ace = pd.DataFrame({'l_aces': l_total})

In [110]:
aces = pd.concat([w_ace, l_ace],axis=1)
aces = aces.fillna(0)
aces = aces.reset_index(level=0)
aces.rename(columns = {'index': 'player_name'}, inplace=True)

aces['avg_aces'] = (aces['w_aces'] + aces['l_aces'])/2
aces.head()

,player_name,w_aces,l_aces,avg_aces
0,Adam Chadaj,1.000000,2.000000,1.500000
1,Adam Pavlasek,9.200000,5.888889,7.544444
2,Adrian Andreev,0.000000,1.500000,0.750000
3,Adrian Garcia,3.000000,2.772727,2.886364
4,Adrian Mannarino,5.677885,3.772549,4.725217


In [111]:
aces

,player_name,w_aces,l_aces,avg_aces
0,Adam Chadaj,1.000000,2.000000,1.500000
1,Adam Pavlasek,9.200000,5.888889,7.544444
2,Adrian Andreev,0.000000,1.500000,0.750000
3,Adrian Garcia,3.000000,2.772727,2.886364
4,Adrian Mannarino,5.677885,3.772549,4.725217
...,...,...,...,...
1439,Zach Dailey,0.000000,5.000000,2.500000
1440,Zbynek Mlynarik,0.000000,4.000000,2.000000
1441,Zdenek Kolar,0.000000,2.000000,1.000000
1442,Zhe Li,0.000000,2.400000,1.200000


In [113]:
allsurfaces_merged

,player_id,player_name,name_first,name_last,hand,dob,ioc,height,total_wins,total_losses,total_matches,perc_of_total_wins,finals,titles,matches,percentage_hard,percentage_carpet,percentage_grass,percentage_clay
0,103819,Roger Federer,Roger,Federer,R,19810808,SUI,185.0,1170,250,1420,82.39,151,102,1420.0,83.35,74.00,87.38,76.92
1,104745,Rafael Nadal,Rafael,Nadal,L,19860603,ESP,185.0,975,198,1173,83.12,121,87,1173.0,78.06,16.67,78.65,91.06
2,104925,Novak Djokovic,Novak,Djokovic,R,19870522,SRB,188.0,927,179,1106,83.82,117,86,1106.0,84.50,55.56,86.32,81.95
3,103970,David Ferrer,David,Ferrer,R,19820402,ESP,175.0,691,364,1055,65.50,52,27,1055.0,63.36,43.75,61.76,69.35
4,104918,Andy Murray,Andy,Murray,R,19870515,GBR,190.0,643,209,852,75.47,64,45,852.0,75.78,72.73,81.60,68.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,105116,Dusan Lojda,Dusan,Lojda,L,19880308,CZE,NaN,0,1,1,0.00,0,0,0.0,0.00,0.00,0.00,0.00
1446,106362,Laurent Lokoli,Laurent,Lokoli,R,19941018,FRA,NaN,0,5,5,0.00,0,0,0.0,0.00,0.00,0.00,0.00
1447,103033,Helder Lopes,Helder,Lopes,R,19770807,POR,NaN,0,1,1,0.00,0,0,0.0,0.00,0.00,0.00,0.00
1448,105782,Enrique Lopez Perez,Enrique,Lopez Perez,R,19910603,ESP,NaN,0,1,1,0.00,0,0,0.0,0.00,0.00,0.00,0.00


In [114]:
aces_ = aces[['player_name', 'avg_aces']]
aces_merg = pd.concat([allsurfaces_merged, aces_])

In [115]:
aces_merg

,player_id,player_name,name_first,name_last,hand,dob,ioc,height,total_wins,total_losses,total_matches,perc_of_total_wins,finals,titles,matches,percentage_hard,percentage_carpet,percentage_grass,percentage_clay,avg_aces
0,103819.0,Roger Federer,Roger,Federer,R,19810808,SUI,185.0,1170.0,250.0,1420.0,82.39,151.0,102.0,1420.0,83.35,74.00,87.38,76.92,NaN
1,104745.0,Rafael Nadal,Rafael,Nadal,L,19860603,ESP,185.0,975.0,198.0,1173.0,83.12,121.0,87.0,1173.0,78.06,16.67,78.65,91.06,NaN
2,104925.0,Novak Djokovic,Novak,Djokovic,R,19870522,SRB,188.0,927.0,179.0,1106.0,83.82,117.0,86.0,1106.0,84.50,55.56,86.32,81.95,NaN
3,103970.0,David Ferrer,David,Ferrer,R,19820402,ESP,175.0,691.0,364.0,1055.0,65.50,52.0,27.0,1055.0,63.36,43.75,61.76,69.35,NaN
4,104918.0,Andy Murray,Andy,Murray,R,19870515,GBR,190.0,643.0,209.0,852.0,75.47,64.0,45.0,852.0,75.78,72.73,81.60,68.84,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,NaN,Zach Dailey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5
1440,NaN,Zbynek Mlynarik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1441,NaN,Zdenek Kolar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1442,NaN,Zhe Li,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2


In [ ]:
# Total Wins and Losses of Players

finals = atpmatches[(atpmatches['round'] == 'F')]
semifinals = atpmatches[(atpmatches['round'] == 'SF')]

#get players who won a title (final)
titles_group = finals.groupby('winner_name').size()
#get players who reach a final match - whether won or lost
finals_group = semifinals.groupby('winner_name').size()

#get all players who won a game or lost a game so as to count wins/losses
w_group = atpmatches.groupby('winner_name').size()
l_group = atpmatches.groupby('loser_name').size()

scores = pd.DataFrame({'Wins': w_group, 'Losses': l_group}).fillna(0)
scores[['Wins', 'Losses']] = scores[['Wins', 'Losses']].astype(int)
scores = scores.reindex(['Wins', 'Losses'], axis=1)

scores['Matches'] = scores['Wins'] + scores['Losses']
scores['Wins(%)'] = np.round(scores['Wins']*100/scores['Matches'],2)

scores.index.name = 'player_name'

scores = scores.join(pd.DataFrame(finals_group, columns = ['Finals'],)).fillna(0)
scores = scores.join(pd.DataFrame(titles_group, columns = ['Titles'],)).fillna(0)
   
    
scores['Titles'] = scores['Titles'].astype('int')
scores['Finals'] = scores['Finals'].astype('int')
    
scores.sort_values(['player_name'], inplace=True)
scores.head()


In [ ]:
print('Players evolved: ', scores.shape[0])

scores = scores.sort_values(['player_name'])
scores.reset_index(level=0, inplace=True)
scores.head()

atpplayers = pd.merge(left=atpplayers, right=scores, left_on='player_name', right_on='player_name')
print('Shape of Dataframe: ', atpplayers.shape[0])
atpplayers.head(6)

atpplayers.drop('total_matches', axis=1, inplace=True)
atpplayers.to_csv('~\Desktop\atp\prod\atpplayers.csv')

In [ ]:
Wins/Losses on specific Tournemant
aka. All player Statistics per Tournament


In [ ]:
def wins_on_tour_per_player(winsOnTour, tourney):
    
    tourney_ = str(tourney)
    matches = winsOnTour[(winsOnTour['tourney_name'] == tourney_)]
    finals = matches[(matches['round'] == 'F')]
    semifinals = matches[(matches['round'] == 'SF')]

    titles_group = finals.groupby('winner_name').size()
    finals_group = semifinals.groupby('winner_name').size()
    
    w_group = matches.groupby('winner_name').size()
    l_group = matches.groupby('loser_name').size()
    
    scores = pd.DataFrame({'Wins': w_group, 'Losses': l_group}).fillna(0)
    scores[['Wins', 'Losses']] = scores[['Wins', 'Losses']].astype(int)
    
    scores = scores.reindex(['Wins', 'Losses'], axis=1)
    
    scores['Matches'] = scores['Wins'] + scores['Losses']
    scores['Wins(%)'] = np.round(scores['Wins']*100/scores['Matches'],2)
    
    scores.index.name = 'player_name'
    
    scores = scores.join(pd.DataFrame(finals_group, columns = ['Finals'],)).fillna(0)
    scores = scores.join(pd.DataFrame(titles_group, columns = ['Titles'],)).fillna(0)
    
    scores['Titles'] = scores['Titles'].astype('int')
    scores['Finals'] = scores['Finals'].astype('int')
    
    return scores

In [ ]:
wins_on_tour_per_player(atpmatches, 'US Open')

In [ ]:
Same Handedness¶

atpmatches.loc[(atpmatches['winner_hand']==atpmatches['loser_hand']), 'handedness'] = 1
atpmatches.loc[((atpmatches['winner_hand']=='R') & (atpmatches['loser_hand']=='L')), 'handedness'] = 2
atpmatches.loc[((atpmatches['winner_hand']=='L') & (atpmatches['loser_hand']=='R')), 'handedness'] = 3